In [75]:

import pandas as pd
import glob
import shutil
import os

In [76]:
completeDf = pd.read_csv("/home/alextay96/Desktop/workspace/dmg_price/data/complete_encoded.csv")
pgCostDf = pd.read_csv("/home/alextay96/Desktop/workspace/dmg_price/pg_cost_sim_1.0.csv")
taskDf = pd.read_csv("/home/alextay96/Desktop/workspace/mrm_workspace/dmg_consistent_detection/scratchpad/tasks.csv")
mergedDf = completeDf.merge(pgCostDf, on="CaseID")



In [77]:
commonVtype = mergedDf.groupby(["Vehicle_Type"])["CaseID"].count().sort_values(ascending=False)
commonVtype

Vehicle_Type
Saloon - 4 Dr            168242
Hatchback - 5 Dr         112351
MPV                       40910
SUV - 5 Dr                39978
Pickup - 4 Dr Dbl.Cab     23861
Van                        2387
Pickup - 2 Dr              1949
Hatchback - 3 Dr           1200
Mini Van                   1185
Coupe - 2 Dr                924
4x4 - 5 Dr                  569
Van - Window Van            536
Prime Mover                 397
Crossover                   277
Pickup - 2 Dr Sgl.Cab       149
Van - Panel                 141
4x4 - 4 Dr                  126
Others - 4 Dr               101
Mini Wagon                   83
4x2 - 5 Dr                   62
Estate - 5 Dr                47
Cabriolet - 2 Dr             40
Jeep                         38
4x4 - 3 Dr                   35
MUV                          33
Station Wagon                31
Tourer                       25
4x2 - 2 Dr                   23
Lorry                        21
Bike - Cub/Moped             21
Saloon - 2 Dr              

In [78]:
targetVehicleType = "Saloon - 4 Dr"
subset = mergedDf[mergedDf["Vehicle_Type"] == targetVehicleType]
balanceFront = subset.groupby(["cost_pg_front"])["CaseID"].count()
balanceRear = subset.groupby(["cost_pg_rear"])["CaseID"].count()

balanceFront

cost_pg_front
0    93134
1    66238
2     8870
Name: CaseID, dtype: int64

In [79]:
remoteDir = "/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img"
viewName = "FrontView"
targetDir = targetVehicleType.replace(" ", "")
srcRemoteDir = f"{remoteDir}/{targetDir}/{viewName}"
allImgRemote = glob.glob(f"{srcRemoteDir}/**/*.JPG",recursive=True)



In [80]:
filesDf = pd.read_csv("/home/alextay96/Desktop/workspace/mrm_workspace/dmg_consistent_detection/scratchpad/selected_raw.csv", low_memory=False)
filesDf["FinalDate"] = pd.to_datetime(filesDf["FinalDate"], errors='coerce')
finalFilesDf = filesDf.sort_values("FinalDate", ascending=True).groupby(["CaseID", "StdDocDesc"]).head(1)
finalFilesDf.to_csv("./final_files.csv")

In [81]:
transports = {}
caseIdAvailable = []
for remotePath in allImgRemote:
    remoteFile = remotePath.split("/")[-1]
    elem = remoteFile.split("_")
    caseId = int(elem[0])
    iDocId = int(elem[1].split(".")[0])
    targets = finalFilesDf[(finalFilesDf["iDOCID"] == iDocId) & (finalFilesDf["CaseID"] == caseId)]
    if(targets.empty):
        continue
    
    transports[caseId] = remotePath
    caseIdAvailable.append(caseId)
transports

{10097796: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/10097796_442212881.JPG',
 10198209: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/10198209_447685032.JPG',
 10199004: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/10199004_447348959.JPG',
 10199391: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/10199391_447610372.JPG',
 10200420: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/10200420_447367536.JPG',
 10200432: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/10200432_447293651.JPG',
 10200444: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/10200444_447431957.JPG',
 10200679: '/run/user/1000/gvfs/smb-share:server=192.168.1.4,share=d$/extract_img/Saloon-4Dr/FrontView/1

In [82]:
foundDf = subset[subset["CaseID"].isin(caseIdAvailable)]
imgDf = foundDf.groupby(["cost_pg_front"])["CaseID"].count()
imgDf

cost_pg_front
0     549
1    1193
2     423
Name: CaseID, dtype: int64

In [83]:
foundDf

,Unnamed: 0.1,Unnamed: 0_x,Model,Make,CaseID,Vehicle_Model,DamageLoss_Type,Reason_Flag,NCB_Stat,Car_Age,...,cost_pg_front,cost_pg_rear,raw_pg_cost_front,raw_pg_cost_rear,raw_pg_cost_front_quantized,raw_pg_cost_rear_quantized,cost_pg_front_left,cost_pg_front_right,cost_pg_rear_left,cost_pg_rear_right
42,53,53,City 2014 On,HONDA,10198209,HONDA City 2014 On 4 Dr Sal 1.5 S AUTO 118Bhp ...,Collision,Payment Settled,0,5.0,...,1,0,15026.25,0.00,5,0,0,0,0,0
165,215,215,City 2014 On,HONDA,10199004,HONDA City 2014 On 4 Dr Sal 1.5 E AUTO 118Bhp ...,Collision,Payment Settled,1,5.0,...,1,0,19191.21,0.00,6,0,0,0,0,0
253,345,345,Cerato 2013 On,KIA,10199391,KIA Cerato 2013 On 4 Dr Sal 1.6 KX AUTO 128Bhp...,Collision,Payment Settled,1,3.0,...,1,0,16316.10,0.00,5,0,0,0,0,0
323,455,455,Saga 2008 On,PROTON,10200420,PROTON Saga 2008 On 4 Dr Sal 1.3 FL (Executive...,Collision,Payment Settled,1,8.0,...,1,0,5515.59,0.00,1,0,0,0,0,0
350,489,489,Almera 2015 On,NISSAN,10200741,NISSAN Almera 2015 On 4 Dr Sal 1.5 E AUTO 101B...,Collision,Payment Settled,1,4.0,...,1,0,12584.00,0.00,4,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394807,562118,562118,Saga 2008 To 2016,PROTON,13076951,PROTON Saga 2008 To 2016 4 Dr Sal 1.30 PLUS AU...,Collision,Payment Settled,1,7.0,...,0,0,480.00,0.00,0,0,0,0,0,0
394943,562303,562303,City 2020 On,HONDA,13074116,HONDA City 2020 On 4 Dr Sal 1.50 1.5 V 119Bhp ...,Collision,NaN,0,1.0,...,0,0,1842.25,2685.93,0,0,0,0,0,0
394998,562380,562380,City 2020 On,HONDA,13074208,HONDA City 2020 On 4 Dr Sal 1.50 1.5 E 119Bhp ...,Collision,Payment Settled,0,2.0,...,1,0,3642.48,0.00,1,0,0,0,0,0
395313,562829,562829,Vios 2007 On,TOYOTA,13079408,TOYOTA Vios 2007 On 4 Dr Sal 1.50 S (NCP93R) A...,Self-Accident,Payment Settled,1,13.0,...,1,0,7288.76,0.00,2,0,0,0,0,0


In [84]:
outBaseDir = "/home/alextay96/Desktop/workspace/mrm_workspace/dmg_consistent_detection/data/level3"
outDir = f"{outBaseDir}/{targetDir}/{viewName}"
os.makedirs(outDir, exist_ok=True)
for label in [0,1,2]:
    levelDir = os.path.join(outDir, str(label))
    os.makedirs(levelDir, exist_ok=True)
    
    for _, row in foundDf[foundDf["cost_pg_front"] == label].iterrows():
        remotePath = transports.get(row["CaseID"], None)
        if(not remotePath ):
            continue
        shutil.copy(remotePath, levelDir)